In [1]:
### Change DIDs for subscribers

In [5]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

Choose connection (prod or lab):prod
You are connected to Reseller: Roamability Test RT.    
Available credit for the Reseller: 3008 USD.

The Reseller contains the following accounts:
 0    Roamability-Porto Test (accountID: 352332)
 1    Telzar Relay test (accountID: 352341)
 2    Roamability - SMPP TEST  (accountID: 352343)
 3    Telzar Relay (accountID: 352344)
 4    Roamability Test Sim-cards (accountID: 352349)
 5    Roamability MIFI Test (Denis) (accountID: 352415)
 6    RB Test hosted GGSN KR (accountID: 352424)
 7    DID's  (accountID: 352441)
 8    RB MiFi test account (accountID: 352446)
 9    soapApiTest (accountID: 352447)
 10   uCloudLink RB Test (accountID: 352448)
 11   Verint  (Roy's test) (accountID: 352471)
 12   Antoloy ( Roy's test) (accountID: 352472)
 13   Sure sample cards (accountID: 352479)
 14   Old Partner RT (accountID: 352527)
 15   Old Partner RT (accountID: 352528)
 16   aaa (accountID: 352529)
 17   Michael D test delete (accountID: 352531)
 18   Sigos 

In [3]:
imsi_list = list(range(515030191000107, 515030191000206+1))
origin_did_list = list(range(639181000107, 639181000206+1))
did_list = ['19284338322']
did_list = origin_did_list

In [4]:
for did in did_list:
    for imsi in imsi_list[:10]:
        subscriber = client.service.getSubscriberByIMSI(imsi)
        if subscriber.result.code == '1' and subscriber.subscriber.MSISDN.startswith('639'):
            mod_subscriber = client.service.modifySubscriberMSISDN(subscriber.subscriber.subscriberId, did)
            if mod_subscriber.result.code == '1':
                print(f'{did}-{imsi}: {subscriber.subscriber.MSISDN} -> {mod_subscriber.subscriber.MSISDN}')
                break
            else:
                print(f'{did}-{imsi}: Error: {mod_subscriber.result.code}, {mod_subscriber.result.description}')
                break
        elif ~subscriber.subscriber.MSISDN.startswith('639'):
            print(f'{did}-{imsi}: Error: current MSISDN: {subscriber.subscriber.MSISDN}')
        else:
            print(f'{did}-{imsi}: Error: {subscriber.result.code}, {subscriber.result.description}')

19284338322-515030191000107: Error: current MSISDN: 16282710936
19284338322-515030191000108: Error: current MSISDN: 19282681139
19284338322-515030191000109: Error: current MSISDN: 19284478361
19284338322-515030191000110: Error: current MSISDN: 19288328609
19284338322-515030191000111: 639181000111 -> 19284338322


In [6]:
# Change back
for did in did_list:
    for imsi in imsi_list[:10]:
        subscriber = client.service.getSubscriberByIMSI(imsi)
        if subscriber.result.code == '1':
            mod_subscriber = client.service.modifySubscriberMSISDN(subscriber.subscriber.subscriberId, did)
            if mod_subscriber.result.code == '1':
                print(f'{did}-{imsi}: {subscriber.subscriber.MSISDN} -> {mod_subscriber.subscriber.MSISDN}')
                break
            else:
                print(f'{did}-{imsi}: Error: {mod_subscriber.result.code}, {mod_subscriber.result.description}')
                break
        elif ~subscriber.subscriber.MSISDN.startswith('639'):
            print(f'{did}-{imsi}: Error: current MSISDN: {subscriber.subscriber.MSISDN}')
        else:
            print(f'{did}-{imsi}: Error: {subscriber.result.code}, {subscriber.result.description}')

639181000107-515030191000107: 639181000107 -> 639181000107
639181000108-515030191000107: 639181000107 -> 639181000108
639181000109-515030191000107: 639181000108 -> 639181000109
639181000110-515030191000107: 639181000109 -> 639181000110
639181000111-515030191000107: 639181000110 -> 639181000111
639181000112-515030191000107: 639181000111 -> 639181000112
639181000113-515030191000107: 639181000112 -> 639181000113
639181000114-515030191000107: Error: 305, MSISDN already exists
639181000115-515030191000107: 639181000113 -> 639181000115
639181000116-515030191000107: Error: 305, MSISDN already exists
639181000117-515030191000107: Error: 305, MSISDN already exists
639181000118-515030191000107: Error: 305, MSISDN already exists
639181000119-515030191000107: Error: 305, MSISDN already exists
639181000120-515030191000107: Error: 305, MSISDN already exists
639181000121-515030191000107: Error: 305, MSISDN already exists
639181000122-515030191000107: Error: 305, MSISDN already exists
639181000123-515

KeyboardInterrupt: 